In [2]:
import json
import os
from pathlib import Path
import sys

import torch

import numpy as np

from compute_f import split_ts_seq, compute_step_positions
from io_f import read_data_file
from visualize_f import visualize_trajectory, visualize_heatmap, save_figure_to_html

In [3]:
floor_data_dir = '../AI6128 Project1/data/site1/F1'
path_data_dir = floor_data_dir + '/path_data_files'
floor_plan_filename = floor_data_dir + '/floor_image.png'
floor_info_filename = floor_data_dir + '/floor_info.json'

In [4]:
def calibrate_magnetic_wifi_ibeacon_to_position(path_file_list):
    mwi_datas = {}
    for path_filename in path_file_list:
        print(f'Processing {path_filename}...')

        path_datas = read_data_file(path_filename)
        acce_datas = path_datas.acce
        magn_datas = path_datas.magn
        ahrs_datas = path_datas.ahrs
        wifi_datas = path_datas.wifi
        ibeacon_datas = path_datas.ibeacon
        posi_datas = path_datas.waypoint

        step_positions = compute_step_positions(acce_datas, ahrs_datas, posi_datas)

        if wifi_datas.size != 0:
            sep_tss = np.unique(wifi_datas[:, 0].astype(float))
            wifi_datas_list = split_ts_seq(wifi_datas, sep_tss)
            for wifi_ds in wifi_datas_list:
                diff = np.abs(step_positions[:, 0] - float(wifi_ds[0, 0]))
                index = np.argmin(diff)
                target_xy_key = tuple(step_positions[index, 1:3])
                if target_xy_key in mwi_datas:
                    mwi_datas[target_xy_key]['wifi'] = np.append(mwi_datas[target_xy_key]['wifi'], wifi_ds, axis=0)
                else:
                    mwi_datas[target_xy_key] = {
                        'magnetic': np.zeros((0, 4)),
                        'wifi': wifi_ds,
                        'ibeacon': np.zeros((0, 3))
                    }

        if ibeacon_datas.size != 0:
            sep_tss = np.unique(ibeacon_datas[:, 0].astype(float))
            ibeacon_datas_list = split_ts_seq(ibeacon_datas, sep_tss)
            for ibeacon_ds in ibeacon_datas_list:
                diff = np.abs(step_positions[:, 0] - float(ibeacon_ds[0, 0]))
                index = np.argmin(diff)
                target_xy_key = tuple(step_positions[index, 1:3])
                if target_xy_key in mwi_datas:
                    mwi_datas[target_xy_key]['ibeacon'] = np.append(mwi_datas[target_xy_key]['ibeacon'], ibeacon_ds, axis=0)
                else:
                    mwi_datas[target_xy_key] = {
                        'magnetic': np.zeros((0, 4)),
                        'wifi': np.zeros((0, 5)),
                        'ibeacon': ibeacon_ds
                    }

        sep_tss = np.unique(magn_datas[:, 0].astype(float))
        magn_datas_list = split_ts_seq(magn_datas, sep_tss)
        for magn_ds in magn_datas_list:
            diff = np.abs(step_positions[:, 0] - float(magn_ds[0, 0]))
            index = np.argmin(diff)
            target_xy_key = tuple(step_positions[index, 1:3])
            if target_xy_key in mwi_datas:
                mwi_datas[target_xy_key]['magnetic'] = np.append(mwi_datas[target_xy_key]['magnetic'], magn_ds, axis=0)
            else:
                mwi_datas[target_xy_key] = {
                    'magnetic': magn_ds,
                    'wifi': np.zeros((0, 5)),
                    'ibeacon': np.zeros((0, 3))
                }

    return mwi_datas

In [5]:
with open(floor_info_filename) as f:
    floor_info = json.load(f)
width_meter = floor_info["map_info"]["width"]
height_meter = floor_info["map_info"]["height"]

path_filenames = list(Path(path_data_dir).resolve().glob("*.txt"))
mwi_datas = calibrate_magnetic_wifi_ibeacon_to_position(path_filenames)

Processing C:\Users\lijia\Desktop\MSAI\Sem A\AI6128\Proj\AI6128 Project1\data\site1\F1\path_data_files\5dd9e7aac5b77e0006b1732b.txt...
Processing C:\Users\lijia\Desktop\MSAI\Sem A\AI6128\Proj\AI6128 Project1\data\site1\F1\path_data_files\5dd9e7abc5b77e0006b1732d.txt...
Processing C:\Users\lijia\Desktop\MSAI\Sem A\AI6128\Proj\AI6128 Project1\data\site1\F1\path_data_files\5dd9e7b29191710006b5705b.txt...
Processing C:\Users\lijia\Desktop\MSAI\Sem A\AI6128\Proj\AI6128 Project1\data\site1\F1\path_data_files\5dd9e7b7c5b77e0006b1732f.txt...
Processing C:\Users\lijia\Desktop\MSAI\Sem A\AI6128\Proj\AI6128 Project1\data\site1\F1\path_data_files\5dd9e7ba9191710006b5705d.txt...
Processing C:\Users\lijia\Desktop\MSAI\Sem A\AI6128\Proj\AI6128 Project1\data\site1\F1\path_data_files\5dd9e7bdc5b77e0006b17331.txt...
Processing C:\Users\lijia\Desktop\MSAI\Sem A\AI6128\Proj\AI6128 Project1\data\site1\F1\path_data_files\5dd9e7bf9191710006b5705f.txt...
Processing C:\Users\lijia\Desktop\MSAI\Sem A\AI6128\Pro

Processing C:\Users\lijia\Desktop\MSAI\Sem A\AI6128\Proj\AI6128 Project1\data\site1\F1\path_data_files\5dd9fd30c5b77e0006b173bc.txt...
Processing C:\Users\lijia\Desktop\MSAI\Sem A\AI6128\Proj\AI6128 Project1\data\site1\F1\path_data_files\5dd9fd37c5b77e0006b173be.txt...
Processing C:\Users\lijia\Desktop\MSAI\Sem A\AI6128\Proj\AI6128 Project1\data\site1\F1\path_data_files\5dd9fd3a9191710006b570d2.txt...
Processing C:\Users\lijia\Desktop\MSAI\Sem A\AI6128\Proj\AI6128 Project1\data\site1\F1\path_data_files\5dd9fd3b9191710006b570d4.txt...
Processing C:\Users\lijia\Desktop\MSAI\Sem A\AI6128\Proj\AI6128 Project1\data\site1\F1\path_data_files\5dd9fd3bc5b77e0006b173c0.txt...
Processing C:\Users\lijia\Desktop\MSAI\Sem A\AI6128\Proj\AI6128 Project1\data\site1\F1\path_data_files\5dd9fd3dc5b77e0006b173c2.txt...
Processing C:\Users\lijia\Desktop\MSAI\Sem A\AI6128\Proj\AI6128 Project1\data\site1\F1\path_data_files\5dd9fd3e9191710006b570d6.txt...
Processing C:\Users\lijia\Desktop\MSAI\Sem A\AI6128\Pro

In [6]:
print(np.array(list(mwi_datas.values())))

[{'magnetic': array([[ 1.5745595e+12,  2.9089355e+01,  5.0399780e+00, -3.0062866e+01],
        [ 1.5745595e+12,  2.9783630e+01,  4.3457030e+00, -3.0062866e+01],
        [ 1.5745595e+12,  2.7008057e+01,  5.0399780e+00, -3.0734253e+01],
        [ 1.5745595e+12,  2.7702332e+01,  4.3457030e+00, -2.9393005e+01],
        [ 1.5745595e+12,  2.8396606e+01,  6.4270020e+00, -2.8723145e+01],
        [ 1.5745595e+12,  2.7008057e+01,  5.0399780e+00, -3.0062866e+01],
        [ 1.5745595e+12,  2.7702332e+01,  5.7327270e+00, -2.8723145e+01],
        [ 1.5745595e+12,  2.7702332e+01,  5.0399780e+00, -2.9393005e+01],
        [ 1.5745595e+12,  2.7008057e+01,  6.4270020e+00, -2.8723145e+01],
        [ 1.5745595e+12,  2.8396606e+01,  9.2025760e+00, -2.8723145e+01],
        [ 1.5745595e+12,  2.8396606e+01,  9.2025760e+00, -2.8723145e+01],
        [ 1.5745595e+12,  2.9089355e+01,  9.8953250e+00, -2.6042175e+01],
        [ 1.5745595e+12,  2.7702332e+01,  1.2670898e+01, -2.6042175e+01],
        [ 1.5745595e+12, 

In [7]:
bssid = []
for m in mwi_datas.values():
    for w in m['wifi']:
        if w[2] not in bssid:
            bssid.append(w[2])
        

In [8]:
print(bssid)

['a8:0c:ca:03:9d:d7', '74:ee:2a:cd:eb:43', 'a8:0c:ca:83:9d:d7', '04:33:89:79:fc:7c', '12:74:9c:2b:13:8f', '1e:74:9c:2b:13:8f', '0a:74:9c:2b:13:8f', '0e:74:9c:2b:13:8f', '06:74:9c:2b:13:8f', '0a:74:9c:2b:62:7b', '16:74:9c:2b:13:8f', '94:d9:b3:f3:6a:d2', '0e:74:9c:2b:62:7b', '12:74:9c:2b:62:7b', '1e:74:9c:2b:62:7b', '06:74:9c:2b:62:7b', '16:74:9c:2b:62:7b', '0a:74:9c:2b:62:7a', '7c:76:30:c7:bf:50', 'a8:0c:ca:14:da:52', '06:74:9c:2b:62:7a', 'e6:cb:ac:96:a3:92', 'a8:0c:ca:15:80:33', '8c:68:c8:bd:81:54', '1a:74:9c:2b:62:7a', '70:af:6a:fe:fe:ed', '00:2f:d9:9c:61:17', '06:74:9c:2b:1a:33', '16:74:9c:2b:1a:33', '0e:74:9c:2b:62:7a', '28:9e:97:2d:d6:34', '12:74:9c:2b:62:7a', '94:d9:b3:f3:68:6f', '48:7d:2e:3c:52:2e', 'a8:0c:ca:05:80:33', 'a8:0c:ca:03:9c:bd', '0a:74:9c:2b:1a:33', '0e:74:9c:2b:1a:33', '12:74:9c:2b:1a:33', '1e:74:9c:2b:1a:33', '12:74:9c:2b:4e:ab', '1e:74:9c:2b:4c:17', '06:74:9c:2b:52:e7', '06:74:9c:2b:43:fb', '0e:74:9c:2b:43:fb', '1e:74:9c:2b:43:fb', '06:74:9c:2b:4e:ab', '18:40:a4:b5

In [9]:
beaconid = []
for m in mwi_datas.values():
    for b in m['ibeacon']:
        if b[1] not in beaconid:
            beaconid.append(b[1])

In [10]:
print(beaconid)

['9195B3AD-A9D0-4500-85FF-9FB0F65A5201_0_0', '616C6970-6179-626F-7869-626561636F6A_49662_37272', '616C6970-6179-626F-7869-626561636F6A_28214_49911', 'FDA50693-A4E2-4FB1-AFCF-C6EB07647825_10073_61418', '6B76E28A-6FA2-48C9-8502-C1DAA388AB2C_64294_18080', '48F8C9EF-AEF9-482D-987F-3752F1C51DA1_22_40969', '6B76E28A-6FA2-48C9-8502-C1DAA388AB2C_45850_14031', '6B76E28A-6FA2-48C9-8502-C1DAA388AB2C_47237_15305', '6B76E28A-6FA2-48C9-8502-C1DAA388AB2C_25440_53472', 'FB349B5F-8000-0080-0010-00003CFE0000_6155_59255', '800DB72B-5C3C-480B-B6F2-E7D5A7388E75_16160_2818', '6B76E28A-6FA2-48C9-8502-C1DAA388AB2C_19018_35270', '3C3493F1-997A-D187-0AD6-FB5A8D4EFA63_30242_6415', '6B76E28A-6FA2-48C9-8502-C1DAA388AB2C_45101_52752', '616C6970-6179-626F-7869-626561636F6A_3824_27065', '07A50693-A4E2-4FB1-AFCF-201707181336_10004_6119', '9195B3AD-A9D0-4500-85FF-9FB0F65A5201_22017_0', '6B76E28A-6FA2-48C9-8502-C1DAA388AB2C_6269_6410', '6B76E28A-6FA2-48C9-8502-C1DAA388AB2C_51635_15211', 'E7FC9D3C-EF01-4B70-B280-2CF6D50F

In [11]:
csvrow = np.concatenate((bssid, beaconid))
print(csvrow)

['a8:0c:ca:03:9d:d7' '74:ee:2a:cd:eb:43' 'a8:0c:ca:83:9d:d7' ...
 '6B76E28A-6FA2-48C9-8502-C1DAA388AB2C_6269_6410'
 '6B76E28A-6FA2-48C9-8502-C1DAA388AB2C_51635_15211'
 'E7FC9D3C-EF01-4B70-B280-2CF6D50FA5CA_13394_63898']


In [12]:
csvrow = csvrow.tolist()

In [13]:
csvrow.append('magnetic')
csvrow.append('x')
csvrow.append('y')

In [14]:
print(csvrow)

['a8:0c:ca:03:9d:d7', '74:ee:2a:cd:eb:43', 'a8:0c:ca:83:9d:d7', '04:33:89:79:fc:7c', '12:74:9c:2b:13:8f', '1e:74:9c:2b:13:8f', '0a:74:9c:2b:13:8f', '0e:74:9c:2b:13:8f', '06:74:9c:2b:13:8f', '0a:74:9c:2b:62:7b', '16:74:9c:2b:13:8f', '94:d9:b3:f3:6a:d2', '0e:74:9c:2b:62:7b', '12:74:9c:2b:62:7b', '1e:74:9c:2b:62:7b', '06:74:9c:2b:62:7b', '16:74:9c:2b:62:7b', '0a:74:9c:2b:62:7a', '7c:76:30:c7:bf:50', 'a8:0c:ca:14:da:52', '06:74:9c:2b:62:7a', 'e6:cb:ac:96:a3:92', 'a8:0c:ca:15:80:33', '8c:68:c8:bd:81:54', '1a:74:9c:2b:62:7a', '70:af:6a:fe:fe:ed', '00:2f:d9:9c:61:17', '06:74:9c:2b:1a:33', '16:74:9c:2b:1a:33', '0e:74:9c:2b:62:7a', '28:9e:97:2d:d6:34', '12:74:9c:2b:62:7a', '94:d9:b3:f3:68:6f', '48:7d:2e:3c:52:2e', 'a8:0c:ca:05:80:33', 'a8:0c:ca:03:9c:bd', '0a:74:9c:2b:1a:33', '0e:74:9c:2b:1a:33', '12:74:9c:2b:1a:33', '1e:74:9c:2b:1a:33', '12:74:9c:2b:4e:ab', '1e:74:9c:2b:4c:17', '06:74:9c:2b:52:e7', '06:74:9c:2b:43:fb', '0e:74:9c:2b:43:fb', '1e:74:9c:2b:43:fb', '06:74:9c:2b:4e:ab', '18:40:a4:b5

In [15]:
print(mwi_datas.keys())
print(len(csvrow))

dict_keys([(80.20884030092255, 93.50212357024748), (78.65300534867832, 93.8980731985203), (76.57995269542144, 94.38254016318979), (75.371765, 94.800575), (75.11878038621505, 94.3920210193359), (75.20369521238766, 92.11338159294698), (75.07001518343837, 90.65518465488076), (78.10398349210023, 90.53199517022642), (80.95584, 89.890015), (81.6815353153706, 91.03916537725286), (81.317215, 93.31349), (80.71761430042582, 93.52507020185313), (77.59558853464691, 92.0302636115281), (75.19962, 91.212906), (74.39242518518252, 92.5520898172806), (80.73444395417941, 93.42081850575954), (79.1586140686775, 93.7235906206752), (76.08729938475742, 94.54193856096042), (75.16788627045052, 93.08959309976625), (75.74028077906186, 90.6791857722236), (76.86606595321906, 90.63374604839052), (79.20535758268194, 90.30436470989811), (81.63518155006666, 89.99157787331912), (81.72400872690083, 90.50317476441711), (81.5843793582978, 91.69813114055057), (81.51432717773477, 92.47814892986587), (81.17858777706428, 93.75

In [16]:
# import csv
# with open('mwidata.csv',newline='', mode='w') as mwidata:
#     mwidata_writer = csv.writer(mwidata, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#     mwidata_writer.writerow(csvrow)
#     for k in mwi_datas.keys():
#         wifi_data_count=[]
#         ibeacon_data_count=[]
#         r = []
#         for i in range(2547):
#             r.append(0)
#         r[2545] = k[0]
#         r[2546] = k[1]
#         magnetic_data = mwi_datas[k]['magnetic']
#         magnetic_s = np.mean(np.sqrt(np.sum(magnetic_data[:, 1:4] ** 2, axis=1)))
#         r[2544] = magnetic_s
        
#         wifi_data = mwi_datas[k]['wifi']
#         for w in wifi_data:
#             for i in range(2524):
#                 if csvrow[i] == w[2]:
#                     isExist = False
#                     if len(wifi_data_count) > 0:
                        
#                         for j in wifi_data_count :
#                             if j['bssid'] == w[2]:
#                                 tmp = float(w[3])
#                                 tmp = (tmp+r[i]*j['count'])/(j['count']+1)
#                                 j['count']+=1
#                                 r[i] = tmp
#                                 isExist = True
#                                 break
#                         if not isExist:
#                             r[i] = int(w[3])
#                             wifi_data_count.append({'bssid':w[2], 'count':1})
#                     else:
#                         r[i] = int(w[3])
#                         wifi_data_count.append({'bssid':w[2], 'count':1})
#                     break
                    
#         ibeacon_data = mwi_datas[k]['ibeacon']
#         for b in ibeacon_data:
#             for i in range(2524, 2544):
#                 if csvrow[i] == b[1]:
#                     isExist = False
#                     if len(ibeacon_data_count) > 0:
                        
#                         for j in ibeacon_data_count:
#                             if j['bssid'] == b[1]:
#                                 tmp = float(b[2])
#                                 tmp = (tmp+r[i]*j['count'])/(j['count']+1)
#                                 j['count']+=1
#                                 r[i] = tmp
#                                 isExist = True
#                                 break
#                         if not isExist:
#                             r[i] = int(b[2])
#                             ibeacon_data_count.append({'bssid':b[1], 'count':1})
#                     else:
#                         r[i] = int(b[2])
#                         ibeacon_data_count.append({'bssid':b[1], 'count':1})
#                     break
#         mwidata_writer.writerow(r)

In [17]:
import csv
with open('mwidata_all.csv',newline='', mode='w') as mwidata:
    mwidata_writer = csv.writer(mwidata, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    mwidata_writer.writerow(csvrow)
    for k in mwi_datas.keys():
        r = []
        for i in range(2547):
            r.append(0)
        r[2545] = k[0]
        r[2546] = k[1]
        magnetic_data = mwi_datas[k]['magnetic']
        magnetic_s = np.mean(np.sqrt(np.sum(magnetic_data[:, 1:4] ** 2, axis=1)))
        r[2544] = magnetic_s
        mwidata_writer.writerow(r)
        
        wifi_data = mwi_datas[k]['wifi']
        for w in wifi_data:
            r = []
            for i in range(2547):
                r.append(0)
            for i in range(2524):
                if csvrow[i] == w[2]:
                    r[i] = int(w[3])
                    mwidata_writer.writerow(r)
                    break
                    
        ibeacon_data = mwi_datas[k]['ibeacon']
        for b in ibeacon_data:
            r = []
            for i in range(2547):
                r.append(0)
            for i in range(2524, 2544):
                if csvrow[i] == b[1]:
                    r[i] = int(b[2])
                    mwidata_writer.writerow(r)
                    break
